### CITY SHORTEST PATH



In [1]:
from itertools import product, combinations
import numpy as np
import networkx as nx
from icecream import ic
from algorithms import *
import os
import csv
from functools import partial
import multiprocessing
import random

In [2]:
def create_problem(
    size: int,
    *,
    density: float = 1.0,
    negative_values: bool = False,
    noise_level: float = 0.0,
    seed: int = 42,
) -> np.ndarray:
    """Problem generator for Lab3"""
    rng = np.random.default_rng(seed)
    map = rng.random(size=(size, 2))
    problem = rng.random((size, size))
    if negative_values:
        problem = problem * 2 - 1
    problem *= noise_level
    for a, b in product(range(size), repeat=2):
        if rng.random() < density:
            problem[a, b] += np.sqrt(
                np.square(map[a, 0] - map[b, 0]) + np.square(map[a, 1] - map[b, 1])
            )
        else:
            problem[a, b] = np.inf
    np.fill_diagonal(problem, 0)

    return (problem * 1_000).round(), map

In [3]:
# Create the problem instance
problem, map_data = create_problem(10, density=0.15, noise_level=10, negative_values=False)
masked = np.ma.masked_array(problem, mask=np.isinf(problem))
G = nx.from_numpy_array(masked, create_using=nx.DiGraph)

SIZE = problem.shape[0]

In [4]:
def get_path_cost(problem_matrix: np.ndarray, path: list) -> float:
    """Calculates the total weight of a path using the problem matrix."""
    if path is None:
        return np.inf
    cost = 0
    for i in range(len(path) - 1):
        u = path[i]
        v = path[i+1]
        cost += problem_matrix[u, v]
    return cost

In [5]:
for s, d in combinations(range(SIZE), 2):
    path = bfs_path(G, s, d)
    cost = get_path_cost(problem, path)
    
    ic(s, d, path, cost)

ic| s: 0, d: 1, path: None, cost: inf
ic| s: 0, d: 2, path: [0, 2], cost: np.float64(10573.0)
ic| s: 0, d: 3, path: [0, 4, 3], cost: np.float64(16797.0)
ic| s: 0, d: 4, path: [0, 4], cost: np.float64(8430.0)
ic| s: 0, d: 5, path: None, cost: inf
ic| s: 0, d: 6, path: [0, 8, 6], cost: np.float64(7235.0)
ic| s: 0, d: 7, path: [0, 7], cost: np.float64(831.0)
ic| s: 0, d: 8, path: [0, 8], cost: np.float64(1977.0)
ic| s: 0, d: 9, path: [0, 2, 9], cost: np.float64(12781.0)
ic| s: 1, d: 2, path: [1, 9, 7, 2], cost: np.float64(19810.0)
ic| s: 1, d: 3, path: None, cost: inf
ic| s: 1, d: 4, path: None, cost: inf
ic| s: 1, d: 5, path: [1, 5], cost: np.float64(2434.0)
ic| s: 1, d: 6, path: [1, 9, 6], cost: np.float64(14621.0)
ic| s: 1, d: 7, path: [1, 9, 7], cost: np.float64(14523.0)
ic| s: 1, d: 8, path: None, cost: inf
ic| s: 1, d: 9, path: [1, 9], cost: np.float64(6771.0)
ic| s: 2, d: 3, path: None, cost: inf
ic| s: 2, d: 4, path: None, cost: inf
ic| s: 2, d: 5, path: None, cost: inf
ic| s: 2, 

In [ ]:
print("Depth-First Search (DFS) results")
for s, d in combinations(range(SIZE), 2):
    path = dfs_path(G, s, d)
    cost = get_path_cost(problem, path)
    
    ic(s, d, path, cost)

--- Depth-First Search (DFS) Results ---


ic| s: 0, d: 1, path: None, cost: inf
ic| s: 0, d: 2, path: [0, 2], cost: np.float64(10573.0)
ic| s: 0, d: 3, path: [0, 4, 3], cost: np.float64(16797.0)
ic| s: 0, d: 4, path: [0, 4], cost: np.float64(8430.0)
ic| s: 0, d: 5, path: None, cost: inf
ic| s: 0, d: 6, path: [0, 2, 9, 6], cost: np.float64(20631.0)
ic| s: 0, d: 7, path: [0, 2, 9, 6, 7], cost: np.float64(25878.0)
ic| s: 0, d: 8, path: [0, 8], cost: np.float64(1977.0)
ic| s: 0, d: 9, path: [0, 2, 9], cost: np.float64(12781.0)
ic| s: 1, d: 2, path: [1, 9, 6, 7, 2], cost: np.float64(25155.0)
ic| s: 1, d: 3, path: None, cost: inf
ic| s: 1, d: 4, path: None, cost: inf
ic| s: 1, d: 5, path: [1, 5], cost: np.float64(2434.0)
ic| s: 1, d: 6, path: [1, 9, 6], cost: np.float64(14621.0)
ic| s: 1, d: 7, path: [1, 9, 6, 7], cost: np.float64(19868.0)
ic| s: 1, d: 8, path: None, cost: inf
ic| s: 1, d: 9, path: [1, 9], cost: np.float64(6771.0)
ic| s: 2, d: 3, path: None, cost: inf
ic| s: 2, d: 4, path: None, cost: inf
ic| s: 2, d: 5, path: None,

In [7]:
for s, d in combinations(range(SIZE), 2):
    path, cost = dijkstra_path(G, s, d)
    ic(s, d, path, cost)

ic| s: 0, d: 1, path: None, cost: inf
ic| s: 0, d: 2, path: [0, 7, 2], cost: 6118.0
ic| s: 0, d: 3, path: [0, 8, 3], cost: 12420.0
ic| s: 0, d: 4, path: [0, 4], cost: 8430.0
ic| s: 0, d: 5, path: None, cost: inf
ic| s: 0, d: 6, path: [0, 8, 6], cost: 7235.0
ic| s: 0, d: 7, path: [0, 7], cost: 831.0
ic| s: 0, d: 8, path: [0, 8], cost: 1977.0
ic| s: 0, d: 9, path: [0, 7, 2, 9], cost: 8326.0
ic| s: 1, d: 2, path: [1, 9, 7, 2], cost: 19810.0
ic| s: 1, d: 3, path: None, cost: inf
ic| s: 1, d: 4, path: None, cost: inf
ic| s: 1, d: 5, path: [1, 5], cost: 2434.0
ic| s: 1, d: 6, path: [1, 9, 6], cost: 14621.0
ic| s: 1, d: 7, path: [1, 9, 7], cost: 14523.0
ic| s: 1, d: 8, path: None, cost: inf
ic| s: 1, d: 9, path: [1, 9], cost: 6771.0
ic| s: 2, d: 3, path: None, cost: inf
ic| s: 2, d: 4, path: None, cost: inf
ic| s: 2, d: 5, path: None, cost: inf
ic| s: 2, d: 6, path: [2, 9, 6], cost: 10058.0
ic| s: 2, d: 7, path: [2, 9, 7], cost: 9960.0
ic| s: 2, d: 8, path: None, cost: inf
ic| s: 2, d: 9, pat

In [ ]:
print("A* Search (Correct: Positive-Weight Graph)")
for s, d in combinations(range(SIZE), 2):
    path, cost = a_star_path(G, s, d, map_data)
    ic(s, d, path, cost)

--- A* Search (Correct: Positive-Weight Graph) ---


ic| s: 0, d: 1, path: None, cost: inf
ic| s: 0, d: 2, path: [0, 7, 2], cost: 6118.0
ic| s: 0, d: 3, path: [0, 8, 3], cost: 12420.0
ic| s: 0, d: 4, path: [0, 4], cost: 8430.0
ic| s: 0, d: 5, path: None, cost: inf
ic| s: 0, d: 6, path: [0, 8, 6], cost: 7235.0
ic| s: 0, d: 7, path: [0, 7], cost: 831.0
ic| s: 0, d: 8, path: [0, 8], cost: 1977.0
ic| s: 0, d: 9, path: [0, 7, 2, 9], cost: 8326.0
ic| s: 1, d: 2, path: [1, 9, 7, 2], cost: 19810.0
ic| s: 1, d: 3, path: None, cost: inf
ic| s: 1, d: 4, path: None, cost: inf
ic| s: 1, d: 5, path: [1, 5], cost: 2434.0
ic| s: 1, d: 6, path: [1, 9, 6], cost: 14621.0
ic| s: 1, d: 7, path: [1, 9, 7], cost: 14523.0
ic| s: 1, d: 8, path: None, cost: inf
ic| s: 1, d: 9, path: [1, 9], cost: 6771.0
ic| s: 2, d: 3, path: None, cost: inf
ic| s: 2, d: 4, path: None, cost: inf
ic| s: 2, d: 5, path: None, cost: inf
ic| s: 2, d: 6, path: [2, 9, 6], cost: 10058.0
ic| s: 2, d: 7, path: [2, 9, 7], cost: 9960.0
ic| s: 2, d: 8, path: None, cost: inf
ic| s: 2, d: 9, pat

In [ ]:
def process_pair(s, d, problem_matrix, graph, map_data):
    """
    Runs ALL 5 algorithms for the given pair (s, d).
    """
    results = []
    
    # run BFS
    path = bfs_path(graph, s, d)
    path_str = 'None' if path is None else str(path)
    cost = get_path_cost(problem_matrix, path)
    cost_str = 'inf' if cost == np.inf else cost
    hops_str = 'None' if path is None else len(path) - 1
    results.append([s, d, "BFS", path_str, cost_str, hops_str])
    
    # run DFS
    path = dfs_path(graph, s, d)
    path_str = 'None' if path is None else str(path)
    cost = get_path_cost(problem_matrix, path)
    cost_str = 'inf' if cost == np.inf else cost
    hops_str = 'None' if path is None else len(path) - 1
    results.append([s, d, "DFS", path_str, cost_str, hops_str])
    
    # run Dijkstra
    path, cost = dijkstra_path(graph, s, d)
    path_str = 'None' if path is None else str(path)
    cost_str = 'inf' if cost == np.inf else cost
    hops_str = 'None' if path is None else len(path) - 1
    results.append([s, d, "Dijkstra", path_str, cost_str, hops_str])
    
    # run A*
    path, cost = a_star_path(graph, s, d, map_data)
    path_str = 'None' if path is None else str(path)
    cost_str = 'inf' if cost == np.inf else cost
    hops_str = 'None' if path is None else len(path) - 1
    results.append([s, d, "A*", path_str, cost_str, hops_str])
    
    # run Bellman-Ford
    try:
        path = nx.bellman_ford_path(graph, s, d, weight='weight')
        cost = nx.path_weight(graph, path, weight='weight')
        path_str = str(path)
        cost_str = cost
        hops_str = len(path) - 1
    except nx.NetworkXNoPath:
        path_str = 'None'
        cost_str = 'inf'
        hops_str = 'None'
    except nx.NetworkXUnbounded:
        path_str = 'None'
        cost_str = '-inf'
        hops_str = 'None'
    results.append([s, d, "Bellman-Ford", path_str, cost_str, hops_str])

    return results

In [ ]:
# function for running all the experiments

def run_experiment():
    sizes = [10, 20, 50, 100, 200, 500, 1000]
    densities = [0.2, 0.5, 0.8, 1.0]
    noise_levels = [0.0, 0.1, 0.5, 0.8]
    negative_values_options = [False, True]

    # How many samples we take.
    # For computational reason the problems with 1000 and density = 0.8 and 1 
    # have been tested with 50 samples
    SAMPLE_LIMIT = 2000
    
    output_dir = "results"
    os.makedirs(output_dir, exist_ok=True)
    
    param_combinations = product(sizes, densities, noise_levels, negative_values_options)
    
    for params in param_combinations:
        size, density, noise, negative = params
        
        filename = f"stats_size_{size}_dens_{density}_noise_{noise}_neg_{negative}.csv"
        filepath = os.path.join(output_dir, filename)
        
        print(f"--- START problem: {filename} ---")
        
        try:
            problem, map_data = create_problem(
                size, density=density, noise_level=noise, negative_values=negative, seed=42
            )
            masked = np.ma.masked_array(problem, mask=np.isinf(problem))
            G = nx.from_numpy_array(masked, create_using=nx.DiGraph)
            
            # implementation of the sampling:
            # If size is small (<= 100), we do exhaustive search (ALL combinations).
            # If size is large (> 100), we sample a fixed number of pairs.
            
            if size <= 100:
                print(f"   Small graph (Size {size}). Testing ALL pairs.")
                pairs_to_process = list(combinations(range(size), 2))
            else:
                print(f"   Large graph (Size {size}). Testing {SAMPLE_LIMIT} sampled pairs.")
                # Efficiently generate 'SAMPLE_LIMIT' unique random pairs
                pairs_to_process = []
                seen_pairs = set()
                
                # This is a safety check: don't sample more than exists
                max_possible = size * (size - 1)
                target_count = min(SAMPLE_LIMIT, max_possible)
                
                while len(pairs_to_process) < target_count:
                    s = random.randint(0, size - 1)
                    d = random.randint(0, size - 1)
                    if s != d and (s, d) not in seen_pairs:
                        pairs_to_process.append((s, d))
                        seen_pairs.add((s, d))

            with open(filepath, 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(["source", "destination", "algorithm", "path", "cost", "hops"])
                
                # loop over the pairs
                for s, d in pairs_to_process:
                    result_rows = process_pair(s, d, problem, G, map_data)
                    writer.writerows(result_rows)
            
            print(f"--- FINISH problem: {filename} ---")
            
        except Exception as e:
            print(f"!!! FAILED problem: {filename}. Error: {e}")

    print("[ EXPERIMENTS COMPLETED! ]")

In [ ]:
run_experiment()